# Overview
The goal of this notebook is to compare state and sector kWh consumption based on the sale of kWh.

Electric Sales - Sourced from EIA and kWh actually consumed

Sectors include - Residential (RES), Commericial (COM), Transporation (TRA), Industrial (IND), and Other (OTH)


Still To Do:
* **Complete**

Other notebooks to reference:
* [Electricity_Sales_EDA]("https://colab.research.google.com/drive/1LP_qEQlQGi64VO5tJs7_86WKwdNUIm8V#scrollTo=ifZsZJ-ECsKk")

# Getting Powerplant Data

# Probably can query this instead of downloading each link below
# https://www.eia.gov/opendata/qb.php?category=1017

* [BioMass](https://atlas.eia.gov/datasets/biomass-2/explore?location=44.619557%2C61.504001%2C3.67&showTable=true)
* [Petroleum](https://atlas.eia.gov/datasets/petroleum-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Other](https://atlas.eia.gov/datasets/other/explore?location=44.619557%2C61.504001%2C3.67)
* [Battery Storage](https://atlas.eia.gov/datasets/battery-storage/explore?location=44.619557%2C61.504001%2C3.67)
* [Hydro Electric](https://atlas.eia.gov/datasets/hydroelectric-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Nuclear](https://atlas.eia.gov/datasets/nuclear/explore?location=44.619557%2C61.504001%2C3.67)
* [Solar](https://atlas.eia.gov/datasets/solar-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Wind](https://atlas.eia.gov/datasets/wind-2/explore?location=44.619557%2C61.504001%2C3.67)
* [Coal](https://atlas.eia.gov/datasets/coal-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Natural Gas](https://atlas.eia.gov/datasets/natural-gas-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Geothermal](https://atlas.eia.gov/datasets/geothermal-1/explore?location=44.619557%2C61.504001%2C3.67)
* [Pumped Storage](https://atlas.eia.gov/datasets/pumped-storage/explore?location=44.619557%2C61.504001%2C3.67)

* [Map](https://atlas.eia.gov/maps/power-plants-1/about)

In [1]:
try:
    from google.colab import drive

    drive.mount('./drive/')

    %cd drive/My \ Drive/Data606_Energy

except:
    print("No Colab Environment")

Mounted at ./drive/
/content/drive/My  Drive/Data606_Energy


In [2]:
import sys
import pandas as pd
import glob
from tqdm.autonotebook import tqdm
sys.path.append("helpers/")

from energygrid import EGRID

from helper_functions import write_csv

import plotly.express as px

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [3]:
energy_grid = EGRID()
energy_grid.get_states()

In [4]:
def combine_like_files(fn,func):

    f_names = glob.glob(fn)

    df = pd.concat(map(func,f_names)).reset_index(drop=True)

    return df


# Retail Electricty Sales in kWh - https://www.eia.gov/opendata/qb.php?category=38&sdid=ELEC.SALES.AL-ALL.M

In [5]:
import requests
import json

with open("creds/eia_creds.json","r") as f:

    creds = json.load(f)

    key = creds.get("key")


In [6]:
def clean_electricity_sales(df):

    #Split Country and State
    df[["country","state"]] = df["geography"].str.split("-",expand=True)

    #Date cleaning
    df_data_split = pd.DataFrame(df['data'].to_list(),columns=["year_month","million_kWh"])

    df_data_split["kWh"] = df_data_split["million_kWh"] / 0.0001

    df_data_split["date"] = pd.to_datetime(df_data_split["year_month"],format='%Y%m').drop(columns=["year_month"])

    #Combine main df and date cleaning
    df_electric_retail_sales = (pd.concat([df,df_data_split],axis=1)
                                .drop(columns=["year_month","geography","iso3166","copyright","data","start","end","units"])
                                )
    df_electric_retail_sales = df_electric_retail_sales.assign(year=df_electric_retail_sales["date"].dt.year)


    return df_electric_retail_sales



In [7]:
for state in tqdm(energy_grid.states):

    response = requests.get(f"http://api.eia.gov/series/?api_key={key}&series_id=ELEC.SALES.{state}-ALL.M").json()

    data_series = response.get("series")[0]

    fn = f"data/electricity_retail_sales/All/{state}_retail_sales.json"

    pd.DataFrame.from_dict(data_series).to_json(fn)

  0%|          | 0/51 [00:00<?, ?it/s]

In [8]:
#Read file
df_total_electric_retail_sales = combine_like_files("data/electricity_retail_sales/All/*_retail_sales.json",pd.read_json)

df_total_electric_retail_sales.sample(10)

,series_id,name,units,f,description,copyright,source,iso3166,geography,start,end,updated,data
5399,ELEC.SALES.MI-ALL.M,Retail sales of electricity : Michigan : all s...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-MI,USA-MI,200101,202107,2021-09-23T14:54:35-0400,"[200311, 8667.19792]"
588,ELEC.SALES.AK-ALL.M,Retail sales of electricity : Alaska : all sec...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-AK,USA-AK,200101,202107,2021-09-23T14:54:35-0400,"[201309, 487.79956]"
1247,ELEC.SALES.CA-ALL.M,Retail sales of electricity : California : all...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-CA,USA-CA,200101,202107,2021-09-23T14:54:35-0400,"[202007, 25901.29962]"
2956,ELEC.SALES.HI-ALL.M,Retail sales of electricity : Hawaii : all sec...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-HI,USA-HI,200101,202107,2021-09-23T14:54:35-0400,"[200108, 879.80314]"
6366,ELEC.SALES.MO-ALL.M,Retail sales of electricity : Missouri : all s...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-MO,USA-MO,200101,202107,2021-09-23T14:54:35-0400,"[200508, 8403.19222]"
8733,ELEC.SALES.OH-ALL.M,Retail sales of electricity : Ohio : all secto...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-OH,USA-OH,200101,202107,2021-09-23T14:54:35-0400,"[201403, 12906.19179]"
10142,ELEC.SALES.SC-ALL.M,Retail sales of electricity : South Carolina :...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-SC,USA-SC,200101,202107,2021-09-23T14:54:35-0400,"[202004, 5154.26049]"
4813,ELEC.SALES.ME-ALL.M,Retail sales of electricity : Maine : all sect...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-ME,USA-ME,200101,202107,2021-09-23T14:54:35-0400,"[201107, 1030.59307]"
4094,ELEC.SALES.KS-ALL.M,Retail sales of electricity : Kansas : all sec...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-KS,USA-KS,200101,202107,2021-09-23T14:54:35-0400,"[200909, 3198.44265]"
5288,ELEC.SALES.MI-ALL.M,Retail sales of electricity : Michigan : all s...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-MI,USA-MI,200101,202107,2021-09-23T14:54:35-0400,"[201302, 8160.81818]"


In [9]:
df_total_electric_sales_clean = clean_electricity_sales(df_total_electric_retail_sales)

In [10]:
px.scatter(df_total_electric_sales_clean,x="date",y="million_kWh",color="state")

# Electricity Sales by Sector - Some states don't have TRA

* **ALL** - ALL Sectors below combined
* **RES** - Residential
* **COM** - Commercial
* **IND** - Industrial
* **TRA** - Transportation
* **OTH** - Other

In [12]:
sectors = ["ALL","RES","COM","IND","TRA","OTH"]
state_sector_kickouts = set()

for sector in tqdm(sectors):

    print(sector)

    for state in tqdm(energy_grid.states):
        try:
            response = requests.get(f"http://api.eia.gov/series/?api_key={key}&series_id=ELEC.SALES.{state}-{sector}.M").json()

            data_series = response.get("series")[0]

            fn = f"data/electricity_retail_sales/{sector}/{state}_retail_sales.json"

            df = pd.DataFrame.from_dict(data_series)

            df["sector"] = sector

            df.to_json(fn)
        
        except TypeError:
            state_sector_kickouts.add((sector,state))

            continue

  0%|          | 0/6 [00:00<?, ?it/s]

ALL


  0%|          | 0/51 [00:00<?, ?it/s]

RES


  0%|          | 0/51 [00:00<?, ?it/s]

...

  0%|          | 0/51 [00:00<?, ?it/s]

TRA


  0%|          | 0/51 [00:00<?, ?it/s]

OTH


  0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
#sector and state combinations not found
write_csv("data/electricity_retail_sales/kickouts.csv",["sector","state"],state_sector_kickouts)

In [ ]:
res = combine_like_files("data/electricity_retail_sales/RES/*.json",pd.read_json)
com = combine_like_files("data/electricity_retail_sales/COM/*.json",pd.read_json)
ind = combine_like_files("data/electricity_retail_sales/IND/*.json",pd.read_json)
tra = combine_like_files("data/electricity_retail_sales/TRA/*.json",pd.read_json)
oth = combine_like_files("data/electricity_retail_sales/OTH/*.json",pd.read_json)

In [ ]:
df_electricity_sales_sector = pd.concat([res,com,ind,tra,oth]).reset_index(drop=True)

In [ ]:
df_electricity_sales_sector

In [ ]:
df_electricity_sales_sector = clean_electricity_sales(df_electricity_sales_sector)

In [ ]:
df_electricity_sales_sector.dtypes

In [ ]:
df_state_date_sector = df_electricity_sales_sector.groupby(["state","date"]).sum().drop(columns="year")

In [ ]:
df_state_date_all = df_total_electric_sales_clean.groupby(["state","date"]).sum().drop(columns="year")

In [ ]:
df_state_date_sector["difference"] = df_state_date_sector["kWh"] - df_state_date_all["kWh"]

## Looks like a difference in rounding, so both data are good to use

In [ ]:
_ = df_state_date_sector["difference"].plot(figsize=(20,5),title="Difference between sector sum and all")